# D106 baby brain


 This notebook contains the code used for running the single color cy5 cell segmentation, using DAPI staining. 

 * path: \\merfish8\merfish8v2\20230805_D103_Myh67_d80KO\DNA_singleCy5\H1_R1\\*zarr
 * microscope: Lemon
 * Experiment: Myh 6/7 D103


# Get the files associated with the experiment

 define a segmentation folder, max_image, etc by refering to CellSegmentationClean.ipynb

In [4]:
import glob
import sys
sys.path.append(r'../')
from ioMicro import *
path = r'\\merfish9.ucsd.edu\volume1\20230908_D106Luo\DNA\H1_B1_2_3\*zarr'
files = np.sort(glob.glob(path))[926:1126]
print(f"total FOVs: {len(files)}")

total FOVs: 200


In [6]:
#files

getting the folders here:

In [7]:
import os
#\\merfish8\merfish8v2\20230801D104Myh67d80\DNA_singleCy5\AnalysisDeconvolveCG\SegmentationDAPI_CG
analysis_folder =r'\\merfish9.ucsd.edu\volume1\20230908_D106Luo\DNA\AnalysisDeconvolveCG'
segm_folder = analysis_folder + os.sep + "SegmentationDAPI_CG"


In [8]:
psf = np.load(r'D:\Carlos\Scripts/psfs/psf_647_Kiwi.npy')
import napari
napari.view_image(psf)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 54.5, 54.5), zoom=4.059090909090909, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(10.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 20.0, 1.0), (0.0, 110.0, 1.0), (0.0, 110.0, 1.0)), current_step=(10, 55, 55), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'psf' at 0x21f29330400>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: '

In [29]:
def compute_mask(model,file,segm_folder,cell_diameter=20,psf = np.load(r'D:\Carlos\Scripts/psfs/psf_647_Kiwi.npy'),
                 deconvolve=True,save=True,max_image = 10000,snorm=0,
                 flat_field_fl = r'D:\Carlos\Scripts/flat_field/Kiwi__med_col_raw3.npz',redo=False):
    if not os.path.exists(segm_folder): os.makedirs(segm_folder)
    save_fl  = segm_folder+os.sep+os.path.basename(file).split('.')[0]+'--'+os.path.basename(os.path.dirname(file))+'--CYTO_segm.npz'
    print("Working on : " + save_fl)
    masks,tensor,cast = None,None,None
    if not os.path.exists(save_fl) or redo:
        #load image
        load = read_im(file)
        #cast image
        cast = np.array(load[-1],dtype=np.float32)#####mistake
        ### Flat field correct
        im_med = np.load(flat_field_fl)['im']
        im_med = cv2.blur(im_med,(20,20))
        cast = cast/im_med*np.median(im_med)
        tensor = cast
        if deconvolve:
            #using psf and getting tensor
            #tensor = full_deconv(cast,s_=500,pad=100,psf=psf,parameters={'method': 'lucy',  'niter': 30},gpu=True,force=False)
            tensor = full_deconv(cast,s_=500,pad=100,psf=psf,parameters={'method': 'wiener',  'beta': 0.01},gpu=True,force=False)
        if snorm>0:
            tensor = norm_slice(cast,snorm)
        #running the cellpose:
        #step 4 is running the cell i was working with last week.
        use_gpu = True
        def blur(tensor,s=100):
            return np.array([cv2.blur(im_,(s,s)) for im_ in tensor],dtype=np.float32)
        from cellpose import models,io,utils
        model = models.Cellpose(gpu=use_gpu, model_type=model)
        #img = tensor[:,::4,::4]
        #img = blur(tensor,2)[::4,::4,::4]
        img = tensor[::4,::4,::4]
        img_final = np.clip(img/max_image,0,1)
    
        masks, flows, styles, diams = model.eval(img_final,z_axis=0, diameter=cell_diameter, channels=[0,0],do_3D=False,normalize=False,flow_threshold=0.75,cellprob_threshold=-5,stitch_threshold=0.5) 
    
        shape = np.array(tensor.shape)
        if save:
            #step 5: saving the mask and shape in the appropriate location
            np.savez_compressed(save_fl,segm = masks,shape = shape)
    return masks,tensor,cast

In [35]:
file = files[180]
masks,tensor,cast = compute_mask("nuclei",file,segm_folder,cell_diameter=20,
                                 deconvolve=True,
                                 save=True,
                                 max_image = 11000,snorm=0,redo=True)

Working on : \\merfish9.ucsd.edu\volume1\20230908_D106Luo\DNA\AnalysisDeconvolveCG\SegmentationDAPI_CG\Conv_zscan__1106--H1_B1_2_3--CYTO_segm.npz


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:02<00:00,  9.74it/s]


In [36]:
import napari
V = napari.view_image(cast)
V.add_image(tensor,contrast_limits=[0,11000])
V.add_labels(resize(masks,tensor.shape))

<Labels layer 'Labels' at 0x223447efbe0>

### Testing

In [26]:
use_gpu = True
max_image = 11000
def blur(tensor,s=100):
    return np.array([cv2.blur(im_,(s,s)) for im_ in tensor],dtype=np.float32)
from cellpose import models,io,utils
model = 'nuclei'
model = models.Cellpose(gpu=True, model_type=model)
#img = tensor[:,::4,::4]
#img = blur(tensor,2)[::4,::4,::4]
img = tensor[::4,::4,::4]
img_final = np.clip(img/max_image,0,1)

masks, flows, styles, diams = model.eval(img_final,z_axis=0, diameter=20, channels=[0,0],do_3D=False,normalize=False,flow_threshold=0.75,cellprob_threshold=-5,stitch_threshold=0.5) 

In [27]:
import napari
V = napari.view_image(cast)
V.add_image(tensor,contrast_limits=[0,11000])
V.add_labels(resize(masks,tensor.shape))

<Labels layer 'Labels' at 0x222b48ae880>

In [33]:
file = files[180]
napari.view_image(read_im(file)[-1])

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1023.5, 1023.5), zoom=0.2583740234375, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(25.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 50.0, 1.0), (0.0, 2048.0, 1.0), (0.0, 2048.0, 1.0)), current_step=(25, 1024, 1024), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x223447e4e20>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.T

In [37]:
np.percentile(tensor,99)

4772.5683740234235

In [38]:
for file in tqdm(files):
    masks,tensor,cast = compute_mask("nuclei",file,segm_folder,cell_diameter=20,
                                 deconvolve=True,
                                 save=True,
                                 max_image = 11000,snorm=0,redo=True)

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

Working on : \\merfish9.ucsd.edu\volume1\20230908_D106Luo\DNA\AnalysisDeconvolveCG\SegmentationDAPI_CG\Conv_zscan__0926--H1_B1_2_3--CYTO_segm.npz



  0%|▍                                                                                 | 1/200 [00:13<45:29, 13.72s/it]

Working on : \\merfish9.ucsd.edu\volume1\20230908_D106Luo\DNA\AnalysisDeconvolveCG\SegmentationDAPI_CG\Conv_zscan__0927--H1_B1_2_3--CYTO_segm.npz



  1%|▊                                                                                 | 2/200 [00:26<43:16, 13.11s/it]

...


 99%|███████████████████████████████████████████████████████████████████████████████▏| 198/200 [40:50<00:21, 10.83s/it]

Working on : \\merfish9.ucsd.edu\volume1\20230908_D106Luo\DNA\AnalysisDeconvolveCG\SegmentationDAPI_CG\Conv_zscan__1124--H1_B1_2_3--CYTO_segm.npz



100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [41:01<00:10, 10.86s/it]

Working on : \\merfish9.ucsd.edu\volume1\20230908_D106Luo\DNA\AnalysisDeconvolveCG\SegmentationDAPI_CG\Conv_zscan__1125--H1_B1_2_3--CYTO_segm.npz



100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [41:12<00:00, 12.36s/it]


In [ ]:
mask = r'\\merfish8\merfish8v2\20230801D104Myh67d80\DNA_singleCy5\AnalysisDeconvolveCG\SegmentationDAPI_CG\Conv_zscan__001--H1_R1--CYTO_segm.npz'

image = read_im(r"\\merfish8\merfish8v2\20230801D104Myh67d80\DNA_singleCy5\")